In [1]:
import pandas as pd
import jams

In [2]:
from tqdm import tqdm

In [3]:
def filter_jam(jam_file, artist='The Beatles', corpus='Isophonics'):
    # Basically just here to throw out the isophonics corpus
    J = jams.load(jam_file, validate=False)
    
    return not (J.file_metadata.search(artist=artist) and J.search(corpus=corpus))

In [4]:
def extract_key(jam_file):
    
    J = jams.load(jam_file, validate=False)
    
    key = J.file_metadata.artist
    if not key:
        key = J.file_metadata.identifiers['echonest_track_id']
        
    return key

In [5]:
def jam_to_audio(jam_file):
    
    return jam_file.replace('references_v2', 'audio').replace('jams', 'mp3')

In [6]:
jams_files = jams.util.find_with_extension('/home/bmcfee/data/eric_chords/references_v2/', 'jams')

In [7]:
hits = {_: filter_jam(_) for _ in tqdm(jams_files)}

In [8]:
keys = {_: extract_key(_) for _ in tqdm(jams_files)}

In [9]:
df = []
for j in hits:
    if not hits[j]:
        continue
    df.append(dict(audio=jam_to_audio(j),
                   jams=j,
                   key=keys[j],
                   orginal=True))
df = pd.DataFrame.from_records(df)

In [10]:
df.to_csv('../scripts/billboard_index.csv', index=False)
